# TAREA 7 MÉTODOS NUMÉRICOS

### Librerías

In [ ]:
import urllib.request as ureq
import pandas as pd
import time 
import io

## Extracción de datos

In [57]:
# ----- Configuración inicial -----
años = list(range(2003, 2025)) # Desde el 2003 al 2024 por el indiceUV que parte desde ese año
estacion = 290004
variables = ["cieloCubiertoAnual", "temperaturaMediaAnual", "indiceUvbMaximoAnual", "humedadAnual"]

mesMap = {
    1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril",
    5: "Mayo", 6: "Junio", 7: "Julio", 8: "Agosto",
    9: "Septiembre", 10: "Octubre", 11: "Noviembre", 12: "Diciembre"
}

idxFechas = pd.date_range(start='2003-01-01', end='2024-12-31', freq='D')
dfCompleto = pd.DataFrame(index=idxFechas)

# ----- Descarga y Procesamiento de datos -----
urlbase = "https://climatologia.meteochile.gob.cl/application/anual"

# Crearemos un diccionario para guardar todas las tablas (DataFrames)
datos_extraidos = {}

for v in variables:
    # Lista para guardar todos los valores de todos los años para esta variable
    lista_series_anuales = []
    
    for a in años:
        url = f"{urlbase}/{v}/{str(estacion)}/{str(a)}"
        
        try:
            # Descargamos el HTML 
            pagina = ureq.urlopen(url)
            datos_html = pagina.read()
            
            # Extraemos la tabla sabiendo que es la primera
            tabla_anual = pd.read_html(datos_html, header=1)[0]
            
            # Limpiamos los nombres de las columnas
            tabla_anual.columns = tabla_anual.columns.str.strip()
            dia_col = [col for col in tabla_anual.columns if "Día" in col][0]
            tabla_anual = tabla_anual.rename(columns={dia_col: "Día"})

            df_long = tabla_anual.melt(id_vars=["Día"], var_name="Mes", value_name=v)

            # Convertimos los valores a numéricos
            df_long['Día'] = pd.to_numeric(df_long['Día'], errors='coerce')
            df_long[v] = pd.to_numeric(df_long[v], errors='coerce')

            # Eliminamos las filas de resumen y datos faltantes
            df_long = df_long.dropna(subset=['Día', v])
            df_long["Año"] = a

            # Creamos una columna de fecha
            df_long['Fecha'] = pd.to_datetime(df_long[['Año', 'Mes', 'Día']].rename(columns={'Año': 'year', 'Mes': 'month', 'Día': 'day'}), errors='coerce')
            df_long = df_long.dropna(subset=['Fecha'])

            # Creamos la serie indexada por fecha
            serie_anual = df_long.set_index('Fecha')[v]

            # Guardamos la serie anual en la lista
            lista_series_anuales.append(serie_anual)

            time.sleep(1)  # Pausa para no saturar el servidor

        except Exception as e:
            print(f"  -> ERROR al procesar el año {a}: {e}")
    
    # Concatenamos todas las series anuales para esta variable
    if lista_series_anuales:
        serie_variable_completa = pd.concat(lista_series_anuales)
        dfCompleto[v] = serie_variable_completa


In [58]:
dfCompleto

,cieloCubiertoAnual,temperaturaMediaAnual,indiceUvbMaximoAnual,humedadAnual
2003-01-01,NaN,NaN,NaN,NaN
2003-01-02,NaN,NaN,NaN,NaN
2003-01-03,NaN,NaN,NaN,NaN
2003-01-04,NaN,NaN,NaN,NaN
2003-01-05,NaN,NaN,NaN,NaN
...,...,...,...,...
2024-12-27,NaN,NaN,NaN,NaN
2024-12-28,NaN,NaN,NaN,NaN
2024-12-29,NaN,NaN,NaN,NaN
2024-12-30,NaN,NaN,NaN,NaN
